In [1]:
#connexion avec une base de données PostgreSQL 
import psycopg2
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import re
from sklearn.metrics import confusion_matrix


try:
    conn = psycopg2.connect(
          user = "postgres",
          password = "efrei888",
          host = "localhost",
          port = "5432",
          database = "Beevora"
    )
    cur = conn.cursor()
    # Afficher la version de PostgreSQL 
    cur.execute("SELECT version();")
    version = cur.fetchone()
    print("Version : ", version,"\n")
    
    #fermeture de la connexion à la base de données
    #cur.close()
    #conn.close()
    #print("La connexion PostgreSQL est fermée")

except (Exception, psycopg2.Error) as error :
    print ("Erreur lors de la connexion à PostgreSQL", error)

Version :  ('PostgreSQL 13.4, compiled by Visual C++ build 1914, 64-bit',) 



In [2]:
def cote_gagnante(df):

    score_team1 = []
    score_team2 = []

    for x in range(len(df)):  
            if re.search('[0-9]*', str(df['final_score'][x])) and re.search('(?<=:)[0-9]*', str(df['final_score'][x])):
            
                m = re.search('[0-9]*',str(df['final_score'][x]))
                score_team1.append(m.group(0))
            
                n = re.search('(?<=:)[0-9]*',str(df['final_score'][x]))
                score_team2.append(n.group(0))
            else : 
            
                score_team1.append(np.nan)
                score_team2.append(np.nan)
                
    df['score_team1'] = score_team1
    df['score_team2'] = score_team2
    
    cote_passe = []

    for k in range(len(df)):
        if (df['field_label'][k] == 'Match nul' and df['score_team1'][k] == df['score_team2'][k]) \
        or (df['field_label'][k] == df['name1'][k] and df['score_team1'][k] > df['score_team2'][k]) \
        or (df['field_label'][k] == df['name2'][k] and df['score_team1'][k] < df['score_team2'][k]):
            cote_passe.append(True)
        else:
            cote_passe.append(False)
    
    df['cote_passe'] = cote_passe
    
    return df

In [3]:
cur.execute("select match, v_odds.date, v_odds.tournament ->> 'id' as tournament_id, tournaments.category as category, tournaments.name, label, field_name, field_label, \
	(winner -> 0) ->> 'id' as winner, odd, matches.teams[1] as team_1, \
	(select name from teams where id = matches.teams[1]) as name1, matches.teams[2] as team_2, \
	(select name from teams where id = matches.teams[2]) as name2, result #>'{Score,0}'->>'value' as final_score, \
	v_odds.bookmaker_id \
from v_odds, matches, tournaments \
where label = 'Résultat du match' \
and matches.id = v_odds.match \
and tournaments.id = (v_odds.tournament ->> 'id')::int \
and v_odds.sport = 1 \
and odd < 20 ")

res = cur.fetchall()
dfall = pd.DataFrame(res)
dfall.columns = ['match', 'date', 'tournament_id', 'category', 'tournament_name', 'label', 'field_name', 'field_label', 'winner', 'odd', 'team1', 'name1', 'team2', 'name2', 'final_score', 'bookmaker']

In [4]:
dfall

,match,date,tournament_id,category,tournament_name,label,field_name,field_label,winner,odd,team1,name1,team2,name2,final_score,bookmaker
0,28624054,2021-09-30 23:30:00+02:00,27665,49,Primera Division,Résultat du match,Home,Colo Colo,3155,1.27,3155.0,Colo Colo,7029.0,Nublense,0:1,8072
1,25822364,2021-09-29 14:00:00+02:00,704,270,Erovnuli Liga,Résultat du match,Home,FC Samgurali Tskaltubo,44217,1.81,44217.0,FC Samgurali Tskaltubo,5419.0,Torpedo Koutaissi,0:0,8072
2,25406638,2021-08-10 18:00:00+02:00,178,92,Premium Liiga,Résultat du match,Home,Flora Tallinn,3346,1.08,3346.0,Flora Tallinn,3343.0,Trans Narva,5:2,30238
3,28624054,2021-09-30 23:30:00+02:00,27665,49,Primera Division,Résultat du match,Home,Colo Colo,3155,1.35,3155.0,Colo Colo,7029.0,Nublense,0:1,-30239
4,28936218,2021-09-30 18:45:00+02:00,679,393,UEFA Ligue Europa,Résultat du match,Home,Fenerbahce,3052,2.45,3052.0,Fenerbahce,3245.0,Olympiakos,0:3,-8072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282193,27772134,2021-11-06 18:50:00+01:00,266,66,Première division,Résultat du match,Away,Beitar Jerusalem FC,5204,3.30,5333.0,Maccabi Petah Tikva FC,5204.0,Beitar Jerusalem FC,0:3,-8072
282194,27663880,2021-11-20 15:00:00+01:00,211,23,Superligue,Résultat du match,Away,Spartak Trnava,2406,1.56,7084.0,FC Vion Zlate Moravce - Vrable,2406.0,Spartak Trnava,0:0,6207
282195,25720750,2021-11-07 22:30:00+01:00,240,165,LigaPro Primera A,Résultat du match,Away,SC Delfin,241468,5.00,5257.0,LDU Quito,241468.0,SC Delfin,1:1,6207
282196,25720750,2021-11-07 22:30:00+01:00,240,165,LigaPro Primera A,Résultat du match,Away,SC Delfin,241468,4.90,5257.0,LDU Quito,241468.0,SC Delfin,1:1,9703


In [5]:
dfall['date'] = dfall['date'].map(pd.to_datetime)

def get_moy(dt): 
    return dt.month 

def get_dow(dt): 
    return dt.weekday() + 1

def get_dom(dt): 
    return dt.day 

def get_hour(dt): 
    return dt.hour

dfall['month'] = dfall['date'].map(get_moy)

dfall['day_of_week'] = dfall['date'].map(get_dow)

dfall['day_of_month'] = dfall['date'].map(get_dom)

dfall['hour'] = dfall['date'].map(get_hour)

In [6]:
cote_gagnante(dfall)

,match,date,tournament_id,category,tournament_name,label,field_name,field_label,winner,odd,...,name2,final_score,bookmaker,month,day_of_week,day_of_month,hour,score_team1,score_team2,cote_passe
0,28624054,2021-09-30 23:30:00+02:00,27665,49,Primera Division,Résultat du match,Home,Colo Colo,3155,1.27,...,Nublense,0:1,8072,9,4,30,23,0,1,False
1,25822364,2021-09-29 14:00:00+02:00,704,270,Erovnuli Liga,Résultat du match,Home,FC Samgurali Tskaltubo,44217,1.81,...,Torpedo Koutaissi,0:0,8072,9,3,29,14,0,0,False
2,25406638,2021-08-10 18:00:00+02:00,178,92,Premium Liiga,Résultat du match,Home,Flora Tallinn,3346,1.08,...,Trans Narva,5:2,30238,8,2,10,18,5,2,True
3,28624054,2021-09-30 23:30:00+02:00,27665,49,Primera Division,Résultat du match,Home,Colo Colo,3155,1.35,...,Nublense,0:1,-30239,9,4,30,23,0,1,False
4,28936218,2021-09-30 18:45:00+02:00,679,393,UEFA Ligue Europa,Résultat du match,Home,Fenerbahce,3052,2.45,...,Olympiakos,0:3,-8072,9,4,30,18,0,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282193,27772134,2021-11-06 18:50:00+01:00,266,66,Première division,Résultat du match,Away,Beitar Jerusalem FC,5204,3.30,...,Beitar Jerusalem FC,0:3,-8072,11,6,6,18,0,3,True
282194,27663880,2021-11-20 15:00:00+01:00,211,23,Superligue,Résultat du match,Away,Spartak Trnava,2406,1.56,...,Spartak Trnava,0:0,6207,11,6,20,15,0,0,False
282195,25720750,2021-11-07 22:30:00+01:00,240,165,LigaPro Primera A,Résultat du match,Away,SC Delfin,241468,5.00,...,SC Delfin,1:1,6207,11,7,7,22,1,1,False
282196,25720750,2021-11-07 22:30:00+01:00,240,165,LigaPro Primera A,Résultat du match,Away,SC Delfin,241468,4.90,...,SC Delfin,1:1,9703,11,7,7,22,1,1,False


In [7]:
dfall.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282198 entries, 0 to 282197
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   match            282198 non-null  int64  
 1   date             282198 non-null  object 
 2   tournament_id    282198 non-null  object 
 3   category         282198 non-null  int64  
 4   tournament_name  282198 non-null  object 
 5   label            282198 non-null  object 
 6   field_name       282198 non-null  object 
 7   field_label      282198 non-null  object 
 8   winner           182015 non-null  object 
 9   odd              282198 non-null  float64
 10  team1            282174 non-null  float64
 11  name1            282127 non-null  object 
 12  team2            282174 non-null  float64
 13  name2            282148 non-null  object 
 14  final_score      272976 non-null  object 
 15  bookmaker        282198 non-null  int64  
 16  month            282198 non-null  int6

In [8]:
l = [-30239, -30238, 29800, 8072]
dfall = dfall[~dfall.bookmaker.isin(l)]
dfall.reset_index(inplace=True)
dfall.drop(['date', 'tournament_name', 'label', 'name1', 'name2', 'field_label', 'final_score', 'score_team1', 'score_team2'], axis=1, inplace=True)

D:\Anaconda\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [9]:
for k in range(len(dfall)):
    if dfall['field_name'][k] == 'Draw':
        dfall['winner'][k] = 999999

<ipython-input-9-5389311611b1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfall['winner'][k] = 999999
D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
dfall = dfall.dropna()
dfall.reset_index(inplace=True)
dfall.drop(['index', 'level_0'], axis=1, inplace=True)
dfall

,match,tournament_id,category,field_name,winner,odd,team1,team2,bookmaker,month,day_of_week,day_of_month,hour,cote_passe
0,25406638,178,92,Home,3346,1.08,3346.0,3343.0,30238,8,2,10,18,True
1,28936218,679,393,Home,3052,2.45,3052.0,3245.0,-8072,9,4,30,18,False
2,28624054,27665,49,Home,3155,1.36,3155.0,7029.0,8392,9,4,30,23,False
3,25720692,240,165,Home,21941,3.30,21941.0,5255.0,5251,10,1,4,2,True
4,28077158,238,44,Home,3026,2.21,3026.0,5136.0,5251,9,7,26,21,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192950,27772134,266,66,Away,5204,3.30,5333.0,5204.0,-8072,11,6,6,18,True
192951,27663880,211,23,Away,2406,1.56,7084.0,2406.0,6207,11,6,20,15,False
192952,25720750,240,165,Away,241468,5.00,5257.0,241468.0,6207,11,7,7,22,False
192953,25720750,240,165,Away,241468,4.90,5257.0,241468.0,9703,11,7,7,22,False


In [11]:
dfall['team1'] = dfall.team1.astype(int)
dfall['team2'] = dfall.team2.astype(int)
dfall['odd'] = dfall.odd.astype(np.float32)
dfall['match'] = dfall.match.astype(int)
dfall['bookmaker'] = dfall.bookmaker.astype(int)
dfall['tournament_id'] = dfall.tournament_id.astype(int)

In [12]:
dfall.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192955 entries, 0 to 192954
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   match          192955 non-null  int32  
 1   tournament_id  192955 non-null  int32  
 2   category       192955 non-null  int64  
 3   field_name     192955 non-null  object 
 4   winner         192955 non-null  object 
 5   odd            192955 non-null  float32
 6   team1          192955 non-null  int32  
 7   team2          192955 non-null  int32  
 8   bookmaker      192955 non-null  int32  
 9   month          192955 non-null  int64  
 10  day_of_week    192955 non-null  int64  
 11  day_of_month   192955 non-null  int64  
 12  hour           192955 non-null  int64  
 13  cote_passe     192955 non-null  bool   
dtypes: bool(1), float32(1), int32(5), int64(5), object(2)
memory usage: 14.9+ MB


In [13]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [14]:
le.fit(dfall['field_name'])
le.classes_

array(['Away', 'Draw', 'Home'], dtype=object)

In [15]:
dfall['field_name'] = le.transform(dfall['field_name'])

In [16]:
le.fit(dfall['cote_passe'])
le.classes_

array([False,  True])

In [17]:
dfall['cote_passe'] = le.transform(dfall['cote_passe'])
dfall['field_name'] = dfall.field_name.astype(int)
dfall['winner'] = dfall.winner.astype(int)

In [18]:
dfall.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192955 entries, 0 to 192954
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   match          192955 non-null  int32  
 1   tournament_id  192955 non-null  int32  
 2   category       192955 non-null  int64  
 3   field_name     192955 non-null  int32  
 4   winner         192955 non-null  int32  
 5   odd            192955 non-null  float32
 6   team1          192955 non-null  int32  
 7   team2          192955 non-null  int32  
 8   bookmaker      192955 non-null  int32  
 9   month          192955 non-null  int64  
 10  day_of_week    192955 non-null  int64  
 11  day_of_month   192955 non-null  int64  
 12  hour           192955 non-null  int64  
 13  cote_passe     192955 non-null  int64  
dtypes: float32(1), int32(7), int64(6)
memory usage: 14.7 MB


In [19]:
dfall

,match,tournament_id,category,field_name,winner,odd,team1,team2,bookmaker,month,day_of_week,day_of_month,hour,cote_passe
0,25406638,178,92,2,3346,1.08,3346,3343,30238,8,2,10,18,1
1,28936218,679,393,2,3052,2.45,3052,3245,-8072,9,4,30,18,0
2,28624054,27665,49,2,3155,1.36,3155,7029,8392,9,4,30,23,0
3,25720692,240,165,2,21941,3.30,21941,5255,5251,10,1,4,2,1
4,28077158,238,44,2,3026,2.21,3026,5136,5251,9,7,26,21,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192950,27772134,266,66,0,5204,3.30,5333,5204,-8072,11,6,6,18,1
192951,27663880,211,23,0,2406,1.56,7084,2406,6207,11,6,20,15,0
192952,25720750,240,165,0,241468,5.00,5257,241468,6207,11,7,7,22,0
192953,25720750,240,165,0,241468,4.90,5257,241468,9703,11,7,7,22,0


In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn import metrics


X = dfall[['match', 'tournament_id','odd', 'category', 'field_name', 'winner', 'team1', 'team2', 'bookmaker', 
           'month', 'day_of_week', 'day_of_month', 'hour']]

y = dfall['cote_passe']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=False)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((96477, 13), (96478, 13), (96477,), (96478,))

In [21]:
from sklearn.model_selection import cross_val_score

# Logistic Regression 

In [22]:
from sklearn.linear_model import LogisticRegression

model2 = LogisticRegression()
model2 = model2.fit(X_train,y_train)
y_pred = model2.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2))

Accuracy: 71.45


In [23]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp

(68934, 0, 27544, 0)

# DecisionTreeClassifier

In [24]:
clf = DecisionTreeClassifier(max_depth=10)

clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2))

Accuracy: 73.49


In [25]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2))

Accuracy: 63.36


In [26]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp

(50188, 18746, 16605, 10939)

In [27]:
print("crossval:", np.mean(cross_val_score(clf, X_train, y_train, cv=10))*100)
print("Precision:", (tp / (tp + fp))*100)
print("Rappel:", (tp / (tp + fn))*100)

crossval: 69.25364904340465
Precision: 36.85026107461681
Rappel: 39.71463839674702


# RandomForestClassifier

In [28]:
dfall['odd'].mean()

3.354654312133789

In [29]:
model = RandomForestClassifier()
model = model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2))

Accuracy: 70.06


In [30]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp

(60833, 8101, 20781, 6763)

In [31]:
print("crossval:", np.mean(cross_val_score(model, X_train, y_train, cv=10))*100)
print("Precision:", (tp / (tp + fp))*100)
print("Rappel:", (tp / (tp + fn))*100)

crossval: 75.6126678036513
Precision: 45.4991926803014
Rappel: 24.55344176590183


# ---------------------------------1 seul bookmaker-----------------------------------------------

In [32]:
cur.execute("select match, v_odds.date, v_odds.tournament ->> 'id' as tournament_id, tournaments.category as category, tournaments.name, label, field_name, field_label, \
	(winner -> 0) ->> 'id' as winner, odd, matches.teams[1] as team_1, \
	(select name from teams where id = matches.teams[1]) as name1, matches.teams[2] as team_2, \
	(select name from teams where id = matches.teams[2]) as name2, result #>'{Score,0}'->>'value' as final_score, \
	v_odds.bookmaker_id \
from v_odds, matches, tournaments \
where label = 'Résultat du match' \
and matches.id = v_odds.match \
and tournaments.id = (v_odds.tournament ->> 'id')::int \
and v_odds.sport = 1 \
and odd < 20 ")

res = cur.fetchall()
dfall = pd.DataFrame(res)
dfall.columns = ['match', 'date', 'tournament_id', 'category', 'tournament_name', 'label', 'field_name', 'field_label', 'winner', 'odd', 'team1', 'name1', 'team2', 'name2', 'final_score', 'bookmaker']

In [33]:
cote_gagnante(dfall)

,match,date,tournament_id,category,tournament_name,label,field_name,field_label,winner,odd,team1,name1,team2,name2,final_score,bookmaker,score_team1,score_team2,cote_passe
0,28624054,2021-09-30 23:30:00+02:00,27665,49,Primera Division,Résultat du match,Home,Colo Colo,3155,1.27,3155.0,Colo Colo,7029.0,Nublense,0:1,8072,0,1,False
1,25822364,2021-09-29 14:00:00+02:00,704,270,Erovnuli Liga,Résultat du match,Home,FC Samgurali Tskaltubo,44217,1.81,44217.0,FC Samgurali Tskaltubo,5419.0,Torpedo Koutaissi,0:0,8072,0,0,False
2,25406638,2021-08-10 18:00:00+02:00,178,92,Premium Liiga,Résultat du match,Home,Flora Tallinn,3346,1.08,3346.0,Flora Tallinn,3343.0,Trans Narva,5:2,30238,5,2,True
3,28624054,2021-09-30 23:30:00+02:00,27665,49,Primera Division,Résultat du match,Home,Colo Colo,3155,1.35,3155.0,Colo Colo,7029.0,Nublense,0:1,-30239,0,1,False
4,28936218,2021-09-30 18:45:00+02:00,679,393,UEFA Ligue Europa,Résultat du match,Home,Fenerbahce,3052,2.45,3052.0,Fenerbahce,3245.0,Olympiakos,0:3,-8072,0,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282193,27772134,2021-11-06 18:50:00+01:00,266,66,Première division,Résultat du match,Away,Beitar Jerusalem FC,5204,3.30,5333.0,Maccabi Petah Tikva FC,5204.0,Beitar Jerusalem FC,0:3,-8072,0,3,True
282194,27663880,2021-11-20 15:00:00+01:00,211,23,Superligue,Résultat du match,Away,Spartak Trnava,2406,1.56,7084.0,FC Vion Zlate Moravce - Vrable,2406.0,Spartak Trnava,0:0,6207,0,0,False
282195,25720750,2021-11-07 22:30:00+01:00,240,165,LigaPro Primera A,Résultat du match,Away,SC Delfin,241468,5.00,5257.0,LDU Quito,241468.0,SC Delfin,1:1,6207,1,1,False
282196,25720750,2021-11-07 22:30:00+01:00,240,165,LigaPro Primera A,Résultat du match,Away,SC Delfin,241468,4.90,5257.0,LDU Quito,241468.0,SC Delfin,1:1,9703,1,1,False


In [34]:
dfall['date'] = dfall['date'].map(pd.to_datetime)

def get_moy(dt): 
    return dt.month 

def get_dow(dt): 
    return dt.weekday() + 1

def get_dom(dt): 
    return dt.day 

def get_hour(dt): 
    return dt.hour

dfall['month'] = dfall['date'].map(get_moy)

dfall['day_of_week'] = dfall['date'].map(get_dow)

dfall['day_of_month'] = dfall['date'].map(get_dom)

dfall['hour'] = dfall['date'].map(get_hour)

In [35]:
dfall = dfall[dfall.bookmaker == 8392]
dfall.reset_index(inplace=True)

for k in range(len(dfall)):
    if dfall['field_name'][k] == 'Draw':
        dfall['winner'][k] = 999999

<ipython-input-35-ac05b8efacd6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfall['winner'][k] = 999999
D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [36]:
dfall.drop(['date', 'tournament_name', 'label', 'name1', 'name2', 'field_label', 'final_score', 'score_team1', 'score_team2'], axis=1, inplace=True)
l = [-30239, -30238, 29800, 8072]

D:\Anaconda\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [37]:
dfall = dfall.dropna()
dfall.reset_index(inplace=True)
dfall.drop(['index', 'bookmaker', 'level_0'], axis=1, inplace=True)

dfall['team1'] = dfall.team1.astype(int)
dfall['team2'] = dfall.team2.astype(int)
dfall['odd'] = dfall.odd.astype(np.float32)
dfall['match'] = dfall.match.astype(int)
dfall['tournament_id'] = dfall.tournament_id.astype(int)

In [38]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

In [39]:
le.fit(dfall['field_name'])
le.classes_

array(['Away', 'Draw', 'Home'], dtype=object)

In [40]:
dfall['field_name'] = le.transform(dfall['field_name'])

In [41]:
le.fit(dfall['cote_passe'])
le.classes_

array([False,  True])

In [42]:
dfall['cote_passe'] = le.transform(dfall['cote_passe'])

In [43]:
dfall['field_name'] = dfall.field_name.astype(int)

In [44]:
dfall

,match,tournament_id,category,field_name,winner,odd,team1,team2,cote_passe,month,day_of_week,day_of_month,hour
0,28624054,27665,49,2,3155,1.36,3155,7029,0,9,4,30,23
1,27975820,27464,12,2,1940,1.75,1940,1944,0,10,1,4,2
2,26243376,169,91,2,3359,1.15,3359,7305,1,10,6,2,17
3,27674100,170,14,2,2028,3.15,2028,25529,0,9,1,27,18
4,25352098,40,9,2,1761,1.51,1761,1768,1,10,7,17,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27090,25720750,240,165,0,241468,4.90,5257,241468,0,11,7,7,22
27091,25822420,704,270,0,5171,1.92,33819,5171,0,11,6,6,17
27092,28623292,199,159,0,136548,1.78,53841,136548,0,11,6,6,13
27093,27883278,35,30,0,2681,2.07,2528,2681,0,11,7,7,15


In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn import metrics

X = dfall[['match', 'tournament_id','odd', 'category', 'field_name', 'winner', 'team1', 'team2', 
           'month', 'day_of_week', 'day_of_month', 'hour']]

y = dfall['cote_passe']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=False)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((16257, 12), (10838, 12), (16257,), (10838,))

# Logistic Regression 

In [46]:
from sklearn.linear_model import LogisticRegression

model2 = LogisticRegression()
model2 = model2.fit(X_train,y_train)
y_pred = model2.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2))

Accuracy: 71.02


In [47]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp

(7697, 0, 3141, 0)

# DecisionTreeClassifier

In [48]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2))

Accuracy: 61.33


In [49]:
clf = DecisionTreeClassifier(max_depth=7)
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2))

Accuracy: 72.77


In [50]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp

(7277, 420, 2531, 610)

In [51]:
print("crossval:", np.mean(cross_val_score(clf, X_train, y_train, cv=10))*100)
print("Precision:", (tp / (tp + fp))*100)
print("Rappel:", (tp / (tp + fn))*100)

crossval: 68.46846816160469
Precision: 59.22330097087378
Rappel: 19.42056669850366


# RandomForestClassifier

In [52]:
model = RandomForestClassifier()
model = model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2))

Accuracy: 69.27


In [53]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp

(6986, 711, 2619, 522)

In [54]:
print("crossval:", np.mean(cross_val_score(model, X_train, y_train, cv=10))*100)
print("Precision:", (tp / (tp + fp))*100)
print("Rappel:", (tp / (tp + fn))*100)

crossval: 66.46946731005772
Precision: 42.33576642335766
Rappel: 16.6189111747851


 # ----------------------------------------matchs nuls------------------------------------------------

In [55]:
cur.execute("select match, v_odds.date, v_odds.tournament ->> 'id' as tournament_id, tournaments.category as category, tournaments.name, label, field_label, \
	odd, matches.teams[1] as team_1, \
	(select name from teams where id = matches.teams[1]) as name1, matches.teams[2] as team_2, \
	(select name from teams where id = matches.teams[2]) as name2, result #>'{Score,0}'->>'value' as final_score, \
	v_odds.bookmaker_id \
from v_odds, matches, tournaments \
where label = 'Résultat du match' \
and matches.id = v_odds.match \
and tournaments.id = (v_odds.tournament ->> 'id')::int \
and v_odds.sport = 1 \
and field_name = 'Draw' \
and odd < 20 ")

res = cur.fetchall()
dfall = pd.DataFrame(res)
dfall.columns = ['match', 'date', 'tournament_id', 'category', 'tournament_name', 'label', 'field_label', 'odd', 'team1', 'name1', 'team2', 'name2', 'final_score', 'bookmaker']


In [56]:
dfall

,match,date,tournament_id,category,tournament_name,label,field_label,odd,team1,name1,team2,name2,final_score,bookmaker
0,29693664,2021-10-10 15:00:00+02:00,23755,4,UEFA Nations League,Résultat du match,Match nul,3.50,4707.0,Italie,4717.0,Belgique,2:1,9703
1,29693664,2021-10-10 15:00:00+02:00,23755,4,UEFA Nations League,Résultat du match,Match nul,3.40,4707.0,Italie,4717.0,Belgique,2:1,8392
2,29693664,2021-10-10 15:00:00+02:00,23755,4,UEFA Nations League,Résultat du match,Match nul,5.60,4707.0,Italie,4717.0,Belgique,2:1,30238
3,29693664,2021-10-10 15:00:00+02:00,23755,4,UEFA Nations League,Résultat du match,Match nul,7.50,4707.0,Italie,4717.0,Belgique,2:1,6317
4,29693664,2021-10-10 15:00:00+02:00,23755,4,UEFA Nations League,Résultat du match,Match nul,5.60,4707.0,Italie,4717.0,Belgique,2:1,-30238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95795,20718035,2021-09-02 21:00:00+02:00,295,4,"Qualifications Coupe du Monde, CONMEBOL",Résultat du match,Match nul,3.85,4722.0,Venezuela,4819.0,Argentine,1:3,-30239
95796,20718035,2021-09-02 21:00:00+02:00,295,4,"Qualifications Coupe du Monde, CONMEBOL",Résultat du match,Match nul,4.13,4722.0,Venezuela,4819.0,Argentine,1:3,6207
95797,20718035,2021-09-02 21:00:00+02:00,295,4,"Qualifications Coupe du Monde, CONMEBOL",Résultat du match,Match nul,3.85,4722.0,Venezuela,4819.0,Argentine,1:3,30238
95798,20718035,2021-09-02 21:00:00+02:00,295,4,"Qualifications Coupe du Monde, CONMEBOL",Résultat du match,Match nul,4.25,4722.0,Venezuela,4819.0,Argentine,1:3,5251


In [57]:
dfall['date'] = dfall['date'].map(pd.to_datetime)

def get_moy(dt): 
    return dt.month 

def get_dow(dt): 
    return dt.weekday() + 1

def get_dom(dt): 
    return dt.day 

def get_hour(dt): 
    return dt.hour - 1 

dfall['month'] = dfall['date'].map(get_moy)

dfall['day_of_week'] = dfall['date'].map(get_dow)

dfall['day_of_month'] = dfall['date'].map(get_dom)

dfall['hour'] = dfall['date'].map(get_hour)

In [58]:
cote_gagnante(dfall)

,match,date,tournament_id,category,tournament_name,label,field_label,odd,team1,name1,...,name2,final_score,bookmaker,month,day_of_week,day_of_month,hour,score_team1,score_team2,cote_passe
0,29693664,2021-10-10 15:00:00+02:00,23755,4,UEFA Nations League,Résultat du match,Match nul,3.50,4707.0,Italie,...,Belgique,2:1,9703,10,7,10,14,2,1,False
1,29693664,2021-10-10 15:00:00+02:00,23755,4,UEFA Nations League,Résultat du match,Match nul,3.40,4707.0,Italie,...,Belgique,2:1,8392,10,7,10,14,2,1,False
2,29693664,2021-10-10 15:00:00+02:00,23755,4,UEFA Nations League,Résultat du match,Match nul,5.60,4707.0,Italie,...,Belgique,2:1,30238,10,7,10,14,2,1,False
3,29693664,2021-10-10 15:00:00+02:00,23755,4,UEFA Nations League,Résultat du match,Match nul,7.50,4707.0,Italie,...,Belgique,2:1,6317,10,7,10,14,2,1,False
4,29693664,2021-10-10 15:00:00+02:00,23755,4,UEFA Nations League,Résultat du match,Match nul,5.60,4707.0,Italie,...,Belgique,2:1,-30238,10,7,10,14,2,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95795,20718035,2021-09-02 21:00:00+02:00,295,4,"Qualifications Coupe du Monde, CONMEBOL",Résultat du match,Match nul,3.85,4722.0,Venezuela,...,Argentine,1:3,-30239,9,4,2,20,1,3,False
95796,20718035,2021-09-02 21:00:00+02:00,295,4,"Qualifications Coupe du Monde, CONMEBOL",Résultat du match,Match nul,4.13,4722.0,Venezuela,...,Argentine,1:3,6207,9,4,2,20,1,3,False
95797,20718035,2021-09-02 21:00:00+02:00,295,4,"Qualifications Coupe du Monde, CONMEBOL",Résultat du match,Match nul,3.85,4722.0,Venezuela,...,Argentine,1:3,30238,9,4,2,20,1,3,False
95798,20718035,2021-09-02 21:00:00+02:00,295,4,"Qualifications Coupe du Monde, CONMEBOL",Résultat du match,Match nul,4.25,4722.0,Venezuela,...,Argentine,1:3,5251,9,4,2,20,1,3,False


In [59]:
dfall.drop(['date', 'tournament_name', 'label', 'name1', 'name2', 'field_label'], axis=1, inplace=True)

In [60]:
l = [-30239, -30238, 29800, 8072]

dfall = dfall[~dfall.bookmaker.isin(l)]
dfall = dfall.dropna()
dfall.reset_index(inplace=True)

dfall.drop(['final_score'], axis=1, inplace=True)
dfall.drop(['index'], axis=1, inplace=True)
dfall.drop(['score_team1', 'score_team2'], axis=1, inplace=True)
dfall

,match,tournament_id,category,odd,team1,team2,bookmaker,month,day_of_week,day_of_month,hour,cote_passe
0,29693664,23755,4,3.50,4707.0,4717.0,9703,10,7,10,14,False
1,29693664,23755,4,3.40,4707.0,4717.0,8392,10,7,10,14,False
2,29693664,23755,4,5.60,4707.0,4717.0,30238,10,7,10,14,False
3,29693664,23755,4,7.50,4707.0,4717.0,6317,10,7,10,14,False
4,29693664,23755,4,3.42,4707.0,4717.0,5251,10,7,10,14,False
...,...,...,...,...,...,...,...,...,...,...,...,...
63009,20718035,295,4,4.20,4722.0,4819.0,8392,9,4,2,20,False
63010,20718035,295,4,4.15,4722.0,4819.0,9703,9,4,2,20,False
63011,20718035,295,4,4.13,4722.0,4819.0,6207,9,4,2,20,False
63012,20718035,295,4,3.85,4722.0,4819.0,30238,9,4,2,20,False


In [61]:
X = dfall[['match', 'tournament_id','odd', 'category', 'team1', 'team2', 'bookmaker', 
           'month', 'day_of_week', 'day_of_month', 'hour']]

y = dfall['cote_passe']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=False)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((37808, 11), (25206, 11), (37808,), (25206,))

# Logistic Regression 

In [62]:
from sklearn.linear_model import LogisticRegression

model2 = LogisticRegression()
model2 = model2.fit(X_train, y_train)

y_pred = model2.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2))

Accuracy: 73.85


In [63]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp

(18615, 0, 6591, 0)

# DecisionTreeClassifier

In [64]:
clf = DecisionTreeClassifier(max_depth=6)

clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2))

Accuracy: 73.03


In [65]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2))

Accuracy: 60.9


In [66]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp

(13097, 5518, 4338, 2253)

In [67]:
print("crossval:", np.mean(cross_val_score(clf, X_train, y_train, cv=10))*100)
print("Precision:", (tp / (tp + fp))*100)
print("Rappel:", (tp / (tp + fn))*100)

crossval: 60.315339577307306
Precision: 28.992407669540597
Rappel: 34.18297678652708


# RandomForestClassifier

In [68]:
model = RandomForestClassifier()
model = model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2))

Accuracy: 72.8


In [69]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp

(17719, 896, 5960, 631)

In [70]:
print("crossval:", np.mean(cross_val_score(model, X_train, y_train, cv=10))*100)
print("Precision:", (tp / (tp + fp))*100)
print("Rappel:", (tp / (tp + fn))*100)

crossval: 71.06700307440852
Precision: 41.32285527177472
Rappel: 9.573661052950994


 # --------------------------------matchs nuls - 1 bookmaker-----------------------------------

In [71]:
cur.execute("select match, v_odds.date, v_odds.tournament ->> 'id' as tournament_id, tournaments.category as category, tournaments.name, label, field_label, \
	odd, matches.teams[1] as team_1, \
	(select name from teams where id = matches.teams[1]) as name1, matches.teams[2] as team_2, \
	(select name from teams where id = matches.teams[2]) as name2, result #>'{Score,0}'->>'value' as final_score, \
	v_odds.bookmaker_id \
from v_odds, matches, tournaments \
where label = 'Résultat du match' \
and matches.id = v_odds.match \
and tournaments.id = (v_odds.tournament ->> 'id')::int \
and v_odds.sport = 1 \
and field_name = 'Draw' \
and odd < 20 ")

res = cur.fetchall()
dfall = pd.DataFrame(res)
dfall.columns = ['match', 'date', 'tournament_id', 'category', 'tournament_name', 'label', 'field_label', 'odd', 'team1', 'name1', 'team2', 'name2', 'final_score', 'bookmaker']


In [72]:
dfall

,match,date,tournament_id,category,tournament_name,label,field_label,odd,team1,name1,team2,name2,final_score,bookmaker
0,29693664,2021-10-10 15:00:00+02:00,23755,4,UEFA Nations League,Résultat du match,Match nul,3.50,4707.0,Italie,4717.0,Belgique,2:1,9703
1,29693664,2021-10-10 15:00:00+02:00,23755,4,UEFA Nations League,Résultat du match,Match nul,3.40,4707.0,Italie,4717.0,Belgique,2:1,8392
2,29693664,2021-10-10 15:00:00+02:00,23755,4,UEFA Nations League,Résultat du match,Match nul,5.60,4707.0,Italie,4717.0,Belgique,2:1,30238
3,29693664,2021-10-10 15:00:00+02:00,23755,4,UEFA Nations League,Résultat du match,Match nul,7.50,4707.0,Italie,4717.0,Belgique,2:1,6317
4,29693664,2021-10-10 15:00:00+02:00,23755,4,UEFA Nations League,Résultat du match,Match nul,5.60,4707.0,Italie,4717.0,Belgique,2:1,-30238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95795,20718035,2021-09-02 21:00:00+02:00,295,4,"Qualifications Coupe du Monde, CONMEBOL",Résultat du match,Match nul,3.85,4722.0,Venezuela,4819.0,Argentine,1:3,-30239
95796,20718035,2021-09-02 21:00:00+02:00,295,4,"Qualifications Coupe du Monde, CONMEBOL",Résultat du match,Match nul,4.13,4722.0,Venezuela,4819.0,Argentine,1:3,6207
95797,20718035,2021-09-02 21:00:00+02:00,295,4,"Qualifications Coupe du Monde, CONMEBOL",Résultat du match,Match nul,3.85,4722.0,Venezuela,4819.0,Argentine,1:3,30238
95798,20718035,2021-09-02 21:00:00+02:00,295,4,"Qualifications Coupe du Monde, CONMEBOL",Résultat du match,Match nul,4.25,4722.0,Venezuela,4819.0,Argentine,1:3,5251


In [73]:
dfall['date'] = dfall['date'].map(pd.to_datetime)

def get_moy(dt): 
    return dt.month 

def get_dow(dt): 
    return dt.weekday() + 1

def get_dom(dt): 
    return dt.day 

def get_hour(dt): 
    return dt.hour - 1 

dfall['month'] = dfall['date'].map(get_moy)

dfall['day_of_week'] = dfall['date'].map(get_dow)

dfall['day_of_month'] = dfall['date'].map(get_dom)

dfall['hour'] = dfall['date'].map(get_hour)

In [74]:
cote_gagnante(dfall)

,match,date,tournament_id,category,tournament_name,label,field_label,odd,team1,name1,...,name2,final_score,bookmaker,month,day_of_week,day_of_month,hour,score_team1,score_team2,cote_passe
0,29693664,2021-10-10 15:00:00+02:00,23755,4,UEFA Nations League,Résultat du match,Match nul,3.50,4707.0,Italie,...,Belgique,2:1,9703,10,7,10,14,2,1,False
1,29693664,2021-10-10 15:00:00+02:00,23755,4,UEFA Nations League,Résultat du match,Match nul,3.40,4707.0,Italie,...,Belgique,2:1,8392,10,7,10,14,2,1,False
2,29693664,2021-10-10 15:00:00+02:00,23755,4,UEFA Nations League,Résultat du match,Match nul,5.60,4707.0,Italie,...,Belgique,2:1,30238,10,7,10,14,2,1,False
3,29693664,2021-10-10 15:00:00+02:00,23755,4,UEFA Nations League,Résultat du match,Match nul,7.50,4707.0,Italie,...,Belgique,2:1,6317,10,7,10,14,2,1,False
4,29693664,2021-10-10 15:00:00+02:00,23755,4,UEFA Nations League,Résultat du match,Match nul,5.60,4707.0,Italie,...,Belgique,2:1,-30238,10,7,10,14,2,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95795,20718035,2021-09-02 21:00:00+02:00,295,4,"Qualifications Coupe du Monde, CONMEBOL",Résultat du match,Match nul,3.85,4722.0,Venezuela,...,Argentine,1:3,-30239,9,4,2,20,1,3,False
95796,20718035,2021-09-02 21:00:00+02:00,295,4,"Qualifications Coupe du Monde, CONMEBOL",Résultat du match,Match nul,4.13,4722.0,Venezuela,...,Argentine,1:3,6207,9,4,2,20,1,3,False
95797,20718035,2021-09-02 21:00:00+02:00,295,4,"Qualifications Coupe du Monde, CONMEBOL",Résultat du match,Match nul,3.85,4722.0,Venezuela,...,Argentine,1:3,30238,9,4,2,20,1,3,False
95798,20718035,2021-09-02 21:00:00+02:00,295,4,"Qualifications Coupe du Monde, CONMEBOL",Résultat du match,Match nul,4.25,4722.0,Venezuela,...,Argentine,1:3,5251,9,4,2,20,1,3,False


In [75]:
dfall.drop(['date', 'tournament_name', 'label', 'name1', 'name2', 'field_label'], axis=1, inplace=True)

In [76]:
l = [-30239, -30238, 29800, 8072]

dfall = dfall[dfall.bookmaker == 8392]
dfall = dfall.dropna()
dfall.reset_index(inplace=True)

dfall.drop(['final_score'], axis=1, inplace=True)
dfall.drop(['index'], axis=1, inplace=True)
dfall.drop(['score_team1', 'score_team2'], axis=1, inplace=True)
dfall

,match,tournament_id,category,odd,team1,team2,bookmaker,month,day_of_week,day_of_month,hour,cote_passe
0,29693664,23755,4,3.40,4707.0,4717.0,8392,10,7,10,14,False
1,24740256,23755,4,3.15,4707.0,4698.0,8392,10,3,6,20,False
2,24740258,23755,4,3.30,4717.0,4481.0,8392,10,4,7,20,False
3,29693666,23755,4,3.20,4698.0,4481.0,8392,10,7,10,19,False
4,27759520,154,386,2.70,6223.0,6226.0,8392,10,6,23,19,True
...,...,...,...,...,...,...,...,...,...,...,...,...
8830,29305292,295,4,3.00,4820.0,4748.0,8392,10,7,10,22,True
8831,20718061,295,4,4.80,4722.0,4748.0,8392,10,4,7,20,False
8832,20718057,295,4,2.97,4725.0,4820.0,8392,10,4,7,20,True
8833,20718063,295,4,6.40,4757.0,4746.0,8392,10,4,7,20,False


In [77]:
X = dfall[['match', 'tournament_id','odd', 'category', 'team1', 'team2', 
           'month', 'day_of_week', 'day_of_month', 'hour']]

y = dfall['cote_passe']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=False)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5301, 10), (3534, 10), (5301,), (3534,))

# Logistic Regression 

In [78]:
from sklearn.linear_model import LogisticRegression

model2 = LogisticRegression()
model2 = model2.fit(X_train, y_train)

y_pred = model2.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2))

Accuracy: 73.88


In [79]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp

(2611, 0, 923, 0)

# DecisionTreeClassifier

In [80]:
clf = DecisionTreeClassifier(max_depth=6)

clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2))

Accuracy: 72.92


In [81]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2))

Accuracy: 60.19


In [82]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp

(1870, 741, 666, 257)

In [83]:
print("crossval:", np.mean(cross_val_score(clf, X_train, y_train, cv=10))*100)
print("Precision:", (tp / (tp + fp))*100)
print("Rappel:", (tp / (tp + fn))*100)

crossval: 58.17652702270546
Precision: 25.751503006012022
Rappel: 27.843986998916577


# RandomForestClassifier

In [84]:
model = RandomForestClassifier()
model = model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2))

Accuracy: 73.06


In [85]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp

(2558, 53, 899, 24)

In [86]:
print("crossval:", np.mean(cross_val_score(model, X_train, y_train, cv=10))*100)
print("Precision:", (tp / (tp + fp))*100)
print("Rappel:", (tp / (tp + fn))*100)

crossval: 69.79742742422627
Precision: 31.16883116883117
Rappel: 2.600216684723727


 # --------------------------------------------Ligue 1---------------------------------------------------

In [87]:
cur.execute("select match, v_odds.date, v_odds.tournament ->> 'id' as tournament_id, label, field_name, field_label, \
	(winner -> 0) ->> 'id' as winner, odd, matches.teams[1] as team_1, \
	(select name from teams where id = matches.teams[1]) as name1, matches.teams[2] as team_2, \
	(select name from teams where id = matches.teams[2]) as name2, result #>'{Score,0}'->>'value' as final_score, \
	v_odds.bookmaker_id \
from v_odds, matches \
where label = 'Résultat du match' \
and matches.id = v_odds.match \
and (v_odds.tournament ->> 'id')::int = 34 \
and v_odds.sport = 1 \
and odd < 20 ")

res = cur.fetchall()
dfall = pd.DataFrame(res)
dfall.columns = ['match', 'date', 'tournament_id', 'label', 'field_name', 'field_label', 'winner', 'odd', 'team1', 'name1', 'team2', 'name2', 'final_score', 'bookmaker']

In [88]:
dfall

,match,date,tournament_id,label,field_name,field_label,winner,odd,team1,name1,team2,name2,final_score,bookmaker
0,28018324,2021-11-07 15:00:00+01:00,34,Résultat du match,Home,AS Saint-Etienne,1678,2.32,1678,AS Saint-Etienne,1680,Clermont Foot,3:2,5251
1,27884810,2021-09-18 17:00:00+02:00,34,Résultat du match,Home,RC Lens,1648,2.44,1648,RC Lens,1643,Lille OSC,1:0,29800
2,27884968,2021-11-26 21:00:00+01:00,34,Résultat du match,Home,RC Lens,1648,1.78,1648,RC Lens,1684,Angers SCO,2:2,8072
3,28018324,2021-11-07 15:00:00+01:00,34,Résultat du match,Home,AS Saint-Etienne,1678,2.40,1678,AS Saint-Etienne,1680,Clermont Foot,3:2,8072
4,27884810,2021-09-18 17:00:00+02:00,34,Résultat du match,Home,RC Lens,1648,2.50,1648,RC Lens,1643,Lille OSC,1:0,6207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5983,27884810,2021-09-18 17:00:00+02:00,34,Résultat du match,Away,Lille OSC,1643,2.98,1648,RC Lens,1643,Lille OSC,1:0,29800
5984,28018324,2021-11-07 15:00:00+01:00,34,Résultat du match,Away,Clermont Foot,1680,2.82,1678,AS Saint-Etienne,1680,Clermont Foot,3:2,5251
5985,27885012,2021-12-05 15:00:00+01:00,34,Résultat du match,Away,FC Nantes,1647,2.55,1656,FC Lorient,1647,FC Nantes,0:1,9703
5986,27884810,2021-09-18 17:00:00+02:00,34,Résultat du match,Away,Lille OSC,1643,3.00,1648,RC Lens,1643,Lille OSC,1:0,8392


In [89]:
dfall['date'] = dfall['date'].map(pd.to_datetime)

def get_moy(dt): 
    return dt.month 

def get_dow(dt): 
    return dt.weekday() + 1

def get_dom(dt): 
    return dt.day 

def get_hour(dt): 
    return dt.hour - 1 

dfall['month'] = dfall['date'].map(get_moy)

dfall['day_of_week'] = dfall['date'].map(get_dow)

dfall['day_of_month'] = dfall['date'].map(get_dom)

dfall['hour'] = dfall['date'].map(get_hour)

In [90]:
cote_gagnante(dfall)

,match,date,tournament_id,label,field_name,field_label,winner,odd,team1,name1,...,name2,final_score,bookmaker,month,day_of_week,day_of_month,hour,score_team1,score_team2,cote_passe
0,28018324,2021-11-07 15:00:00+01:00,34,Résultat du match,Home,AS Saint-Etienne,1678,2.32,1678,AS Saint-Etienne,...,Clermont Foot,3:2,5251,11,7,7,14,3,2,True
1,27884810,2021-09-18 17:00:00+02:00,34,Résultat du match,Home,RC Lens,1648,2.44,1648,RC Lens,...,Lille OSC,1:0,29800,9,6,18,16,1,0,True
2,27884968,2021-11-26 21:00:00+01:00,34,Résultat du match,Home,RC Lens,1648,1.78,1648,RC Lens,...,Angers SCO,2:2,8072,11,5,26,20,2,2,False
3,28018324,2021-11-07 15:00:00+01:00,34,Résultat du match,Home,AS Saint-Etienne,1678,2.40,1678,AS Saint-Etienne,...,Clermont Foot,3:2,8072,11,7,7,14,3,2,True
4,27884810,2021-09-18 17:00:00+02:00,34,Résultat du match,Home,RC Lens,1648,2.50,1648,RC Lens,...,Lille OSC,1:0,6207,9,6,18,16,1,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5983,27884810,2021-09-18 17:00:00+02:00,34,Résultat du match,Away,Lille OSC,1643,2.98,1648,RC Lens,...,Lille OSC,1:0,29800,9,6,18,16,1,0,False
5984,28018324,2021-11-07 15:00:00+01:00,34,Résultat du match,Away,Clermont Foot,1680,2.82,1678,AS Saint-Etienne,...,Clermont Foot,3:2,5251,11,7,7,14,3,2,False
5985,27885012,2021-12-05 15:00:00+01:00,34,Résultat du match,Away,FC Nantes,1647,2.55,1656,FC Lorient,...,FC Nantes,0:1,9703,12,7,5,14,0,1,True
5986,27884810,2021-09-18 17:00:00+02:00,34,Résultat du match,Away,Lille OSC,1643,3.00,1648,RC Lens,...,Lille OSC,1:0,8392,9,6,18,16,1,0,False


In [91]:
l = [-30239, -30238, 29800, 8072]
dfall = dfall[~dfall.bookmaker.isin(l)]
dfall.reset_index(inplace=True)
dfall.drop(['date', 'label', 'name1', 'name2', 'field_label', 'final_score', 'score_team1', 'score_team2'], axis=1, inplace=True)

D:\Anaconda\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [92]:
for k in range(len(dfall)):
    if dfall['field_name'][k] == 'Draw':
        dfall['winner'][k] = 999999

<ipython-input-92-5389311611b1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfall['winner'][k] = 999999
D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [93]:
dfall = dfall.dropna()
dfall.reset_index(inplace=True)
dfall.drop(['index'], axis=1, inplace=True)
dfall

,level_0,match,tournament_id,field_name,winner,odd,team1,team2,bookmaker,month,day_of_week,day_of_month,hour,cote_passe
0,0,28018324,34,Home,1678,2.32,1678,1680,5251,11,7,7,14,True
1,1,27884810,34,Home,1648,2.50,1648,1643,6207,9,6,18,16,True
2,2,27884768,34,Home,1680,1.71,1680,1651,8392,8,7,29,14,False
3,3,28018320,34,Home,1682,9.50,1682,1653,30238,11,7,7,14,False
4,4,27884810,34,Home,1648,2.50,1648,1643,9703,9,6,18,16,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4099,4099,27884968,34,Away,1684,4.00,1648,1684,5251,11,5,26,20,False
4100,4100,28018324,34,Away,1680,2.82,1678,1680,5251,11,7,7,14,False
4101,4101,27885012,34,Away,1647,2.55,1656,1647,9703,12,7,5,14,True
4102,4102,27884810,34,Away,1643,3.00,1648,1643,8392,9,6,18,16,False


In [94]:
dfall['team1'] = dfall.team1.astype(int)
dfall['team2'] = dfall.team2.astype(int)
dfall['odd'] = dfall.odd.astype(np.float32)
dfall['match'] = dfall.match.astype(int)
dfall['bookmaker'] = dfall.bookmaker.astype(int)
dfall['tournament_id'] = dfall.tournament_id.astype(int)

In [95]:
dfall.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4104 entries, 0 to 4103
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   level_0        4104 non-null   int64  
 1   match          4104 non-null   int32  
 2   tournament_id  4104 non-null   int32  
 3   field_name     4104 non-null   object 
 4   winner         4104 non-null   object 
 5   odd            4104 non-null   float32
 6   team1          4104 non-null   int32  
 7   team2          4104 non-null   int32  
 8   bookmaker      4104 non-null   int32  
 9   month          4104 non-null   int64  
 10  day_of_week    4104 non-null   int64  
 11  day_of_month   4104 non-null   int64  
 12  hour           4104 non-null   int64  
 13  cote_passe     4104 non-null   bool   
dtypes: bool(1), float32(1), int32(5), int64(5), object(2)
memory usage: 324.8+ KB


In [96]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [97]:
le.fit(dfall['field_name'])
le.classes_

array(['Away', 'Draw', 'Home'], dtype=object)

In [98]:
dfall['field_name'] = le.transform(dfall['field_name'])

In [99]:
le.fit(dfall['cote_passe'])
le.classes_

array([False,  True])

In [100]:
dfall['cote_passe'] = le.transform(dfall['cote_passe'])
dfall['field_name'] = dfall.field_name.astype(int)
dfall['winner'] = dfall.winner.astype(int)

In [101]:
dfall.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4104 entries, 0 to 4103
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   level_0        4104 non-null   int64  
 1   match          4104 non-null   int32  
 2   tournament_id  4104 non-null   int32  
 3   field_name     4104 non-null   int32  
 4   winner         4104 non-null   int32  
 5   odd            4104 non-null   float32
 6   team1          4104 non-null   int32  
 7   team2          4104 non-null   int32  
 8   bookmaker      4104 non-null   int32  
 9   month          4104 non-null   int64  
 10  day_of_week    4104 non-null   int64  
 11  day_of_month   4104 non-null   int64  
 12  hour           4104 non-null   int64  
 13  cote_passe     4104 non-null   int64  
dtypes: float32(1), int32(7), int64(6)
memory usage: 320.8 KB


In [102]:
dfall

,level_0,match,tournament_id,field_name,winner,odd,team1,team2,bookmaker,month,day_of_week,day_of_month,hour,cote_passe
0,0,28018324,34,2,1678,2.32,1678,1680,5251,11,7,7,14,1
1,1,27884810,34,2,1648,2.50,1648,1643,6207,9,6,18,16,1
2,2,27884768,34,2,1680,1.71,1680,1651,8392,8,7,29,14,0
3,3,28018320,34,2,1682,9.50,1682,1653,30238,11,7,7,14,0
4,4,27884810,34,2,1648,2.50,1648,1643,9703,9,6,18,16,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4099,4099,27884968,34,0,1684,4.00,1648,1684,5251,11,5,26,20,0
4100,4100,28018324,34,0,1680,2.82,1678,1680,5251,11,7,7,14,0
4101,4101,27885012,34,0,1647,2.55,1656,1647,9703,12,7,5,14,1
4102,4102,27884810,34,0,1643,3.00,1648,1643,8392,9,6,18,16,0


In [103]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn import metrics


X = dfall[['match', 'tournament_id','odd', 'category', 'field_name', 'winner', 'team1', 'team2', 'bookmaker', 
           'month', 'day_of_week', 'day_of_month', 'hour']]

y = dfall['cote_passe']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=False)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

KeyError: "['category'] not in index"

# Logistic Regression 

In [ ]:
from sklearn.linear_model import LogisticRegression

model2 = LogisticRegression()
model2 = model2.fit(X_train,y_train)
y_pred = model2.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2))

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp

# DecisionTreeClassifier

In [ ]:
clf = DecisionTreeClassifier(max_depth=10)

clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2))

In [ ]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2))

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp

In [ ]:
print("crossval:", np.mean(cross_val_score(clf, X_train, y_train, cv=10))*100)
print("Precision:", (tp / (tp + fp))*100)
print("Rappel:", (tp / (tp + fn))*100)

# RandomForestClassifier

In [ ]:
dfall['odd'].mean()

In [ ]:
model = RandomForestClassifier()
model = model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2))

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp

In [ ]:
print("crossval:", np.mean(cross_val_score(model, X_train, y_train, cv=10))*100)
print("Precision:", (tp / (tp + fp))*100)
print("Rappel:", (tp / (tp + fn))*100)